In [ ]:
import math
import os.path

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import activations

from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import time
from skimage import transform, metrics
from umap import UMAP
import datetime
from scipy.signal import argrelextrema
import os
import pandas as pd
import scipy
from skimage import io
import csv
from sklearn.manifold import TSNE
from skimage.metrics import structural_similarity as ssim
import math
import plotly.express as px

from src.nn import ConvCVAE, Decoder, Encoder
from src.nn import RSU7, RSU6, RSU5, RSU4, RSU4F, ConvBlock
from src.nn_utils import SaveImageCallback

gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
np.prod((64, 64))

In [ ]:
label_dim = 40
image_dim = [64, 64, 3]
latent_dim = 128
beta = 0.65


In [ ]:
learning_rate = 0.001
train_size = 0.01
batch_size = 32

# Model
encoder = Encoder(latent_dim)
decoder = Decoder()
model = ConvCVAE(
    encoder,
    decoder,
    label_dim=label_dim,
    latent_dim=latent_dim,
    beta=beta,
    image_dim=image_dim)

# Optiizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
image_shape = (1024, 1024, 1)
latent_dim = 200

inputs = tf.keras.Input(shape=image_shape)
encoder = Encoder(latent_dim, label_dim=5)
out = encoder(inputs)

model = tf.keras.Model(inputs, out)

In [ ]:
model.summary()

In [ ]:
class Encoder(tf.keras.Model):

    def __init__(self, latent_dim, label_dim):
        super(Encoder, self).__init__()

        self.latent_dim = latent_dim
        self.label_dim = label_dim

        self.stage1 = RSU7(16, 64)
        self.pool12 = layers.MaxPool2D((2, 2), 2)

        self.stage2 = RSU6(32, 64)
        self.pool23 = layers.MaxPool2D((2, 2), 2)

        self.stage3 = RSU5(64, 128)
        self.pool34 = layers.MaxPool2D((2, 2), 2)

        self.stage4 = RSU4(128, 256)
        self.pool45 = layers.MaxPool2D((2, 2), 2)
        #out_w_h=64

        self.stage5 = RSU4F(256, 512)
        self.pool56 = layers.MaxPool2D((2, 2), 2)

    def __call__(self, inputs):
        # Encoder block 1

        input_image, input_label = inputs

        hx1 = self.stage1(inputs[0])

        hx = self.pool12(hx1)
        hx2 = self.stage2(hx)
        hx = self.pool23(hx2)

        hx3 = self.stage3(hx)
        hx = self.pool34(hx3)

        hx4 = self.stage4(hx)
        hx = self.pool45(hx4)

        hx5 = self.stage5(hx)
        x = self.pool56(hx5)
        global_pool = layers.GlobalAveragePooling2D()(x)

        x = tf.keras.layers.Flatten()(global_pool)
        x = tf.keras.layers.Dense(512)(x)
        x = layers.LeakyReLU()(x)

        x = tf.keras.layers.Dense(self.latent_dim * 2)(x)
        x = layers.LeakyReLU()(x)

        x_label = layers.Dense(128)(global_pool)
        x_label = layers.LeakyReLU()(x_label)
        x_label = layers.Dense(self.label_dim, name='encoder_label_output')(x_label)
        x_label = layers.LeakyReLU()(x_label)

        return x, x_label


In [ ]:
latent_dim = (200,)
label_dim = (5,)

In [ ]:
encoder = Encoder(latent_dim=200, label_dim=5)
image_input = layers.Input(shape=(1024, 1024, 1))
label_input = layers.Input(shape=(5,))

outputs = encoder([image_input, label_input])
model = tf.keras.Model(inputs=[image_input, label_input], outputs=outputs)

In [ ]:
model.summary()

In [ ]:
class Decoder(tf.keras.Model):

    def __init__(self, label_dim, batch_size=32, out_ch=1):
        super(Decoder, self).__init__()

        self.batch_size = batch_size
        self.label_dim = label_dim

        self.stage6 = RSU4F(256, 512)
        self.stage5d = RSU4F(256, 512)
        self.stage4d = RSU4(128, 256)
        self.stage3d = RSU5(64, 128)
        self.stage2d = RSU6(32, 64)
        self.stage1d = RSU7(16, out_ch)

        self.upsample_1 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')
        self.upsample_2 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')
        self.upsample_3 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')
        self.upsample_4 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')
        self.upsample_5 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')
        self.upsample_6 = layers.UpSampling2D(size=(2, 2), interpolation='bilinear', name='rec_image_output')

    def __call__(self, z_inputs):
        # Reshape input
        # z_image_v, labels = tf.split(z_inputs, axis=1, num_or_size_splits=2)
        z_data = tf.concat(z_inputs, axis=1)
        x = layers.Dense(256)(z_data)
        x = tf.nn.leaky_relu(x)

        x = layers.Reshape(target_shape=(16, 16, 1))(x)

        hx6 = self.stage6(x)
        hx6up = self.upsample_6(hx6)

        max_pool = layers.GlobalAveragePooling2D()(hx6up)
        x_label = layers.Dense(128)(max_pool)
        x_label = layers.LeakyReLU()(x_label)
        x_label = layers.Dense(self.label_dim, name='rec_label_output')(x_label)
        x_label = layers.LeakyReLU()(x_label)

        hx5d = self.stage5d(hx6up)
        hx5dup = self.upsample_5(hx5d)

        hx4d = self.stage4d(hx5dup)
        hx4dup = self.upsample_4(hx4d)

        hx3d = self.stage3d(hx4dup)
        hx3dup = self.upsample_3(hx3d)

        hx2d = self.stage2d(hx3dup)
        hx2dup = self.upsample_2(hx2d)

        hx1d = self.stage1d(hx2dup)
        x = self.upsample_1(hx1d)

        x = activations.sigmoid(x)

        return x, x_label

In [ ]:
latent_dim = (200,)
label_dim = (5,)

In [ ]:
decoder = Decoder(label_dim=5)
z_input = layers.Input(shape=latent_dim)
label_input = layers.Input(shape=label_dim)
outputs = decoder([z_input, label_input])

In [ ]:
model = tf.keras.Model(inputs=[z_input, label_input], outputs=outputs)
model.summary()

In [ ]:

class ConvCVAE(tf.keras.Model):

    def __init__(self,
                 encoder,
                 decoder,
                 label_dim,
                 latent_dim,
                 image_dim,
                 beta=1,
                 ):
        super(ConvCVAE, self).__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.label_dim = label_dim
        self.latent_dim = latent_dim
        self.beta = beta
        self.image_dim = image_dim

    def __call__(self, inputs):
        input_img, input_label = inputs

        z_latent, enc_labels = self.encoder(inputs=(input_img, input_label))
        z_mean, z_log_var = tf.split(z_latent, num_or_size_splits=2, axis=1)
        z_cond = self.reparametrization(z_mean, z_log_var)
        logits, dec_labels = self.decoder((z_cond, input_label))

        recon_img = tf.nn.sigmoid(logits)

        # return recon_img,enc_labels,dec_labels

        # Loss computation #
        latent_loss = - 0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var),
                                            axis=-1)  # KL divergence

        # очень странная метрика для изображений
        # np.prod((64, 64)) *
        reconstr_loss = tf.keras.losses.MSE(tf.keras.backend.flatten(input_img),
                                            tf.keras.backend.flatten(
                                                recon_img))
        enc_label_loss = tf.losses.MSE(tf.keras.backend.flatten(input_label), tf.keras.backend.flatten(enc_labels)
                                       )
        dec_label_loss = tf.losses.MSE(tf.keras.backend.flatten(input_label), tf.keras.backend.flatten(dec_labels)
                                       )
        loss = reconstr_loss + enc_label_loss + dec_label_loss + self.beta * latent_loss  # weighted ELBO loss
        loss = tf.reduce_mean(loss)

        return {
            'recon_img': recon_img,
            'latent_loss': latent_loss,
            'reconstr_loss': reconstr_loss,
            'enc_label_loss': enc_label_loss,
            'dec_label_loss': dec_label_loss,
            'loss': loss,
            'z_mean': z_mean,
            'z_log_var': z_log_var
        }

    def reparametrization(self, z_mean, z_log_var):
        """ Performs the riparametrization trick"""

        eps = tf.random.normal(shape=(self.latent_dim,), mean=0.0, stddev=1.0)
        z = z_mean + tf.math.exp(z_log_var * .5) * eps
        # z_cond = tf.concat([z, input_label], axis=1)  # (batch_size, label_dim + latent_dim)

        return z

In [ ]:
latent_dim = 200
label_dim = 5
image_dim = (1024, 1024, 1)
beta=0.6

encoder = Encoder(label_dim=label_dim, latent_dim=latent_dim)
decoder = Decoder(label_dim=label_dim)

cvae = ConvCVAE(encoder, decoder, label_dim, latent_dim, image_dim=image_dim)

In [ ]:
image_input = layers.Input(shape=(1024, 1024, 1))
label_input = layers.Input(shape=(5,))

outputs = cvae([image_input, label_input])
model = tf.keras.Model(inputs=[image_input, label_input], outputs=outputs)

In [ ]:
model.summary()

In [ ]:
# model=tf.keras.applications.VGG19(include_top=False,input_shape=(64,64,3))
# tf.keras.utils.plot_model(model, to_file='cvae_1.png')

In [ ]:
# Checkpoint path
checkpoint_root = "./CVAE{}_{}_checkpoint".format(latent_dim, beta)
checkpoint_name = "model"
save_prefix = os.path.join(checkpoint_root, checkpoint_name)

# Define the checkpoint
checkpoint = tf.train.Checkpoint(module=model)

In [ ]:
images=np.load('data/saved np/images_no_filters.npy')/255

In [80]:
images.shape

(5, 90, 1024, 1024, 1)

In [ ]:
train_losses = []
train_recon_errors = []
train_latent_losses = []
loss = []
reconstruct_loss = []
latent_loss = []

step_index = 0
n_batches = int(dataset.train_size / batch_size)
n_epochs = 30

print("Number of epochs: {},  number of batches: {}".format(n_epochs, n_batches))

# Epochs Loop
for epoch in range(5):
    start_time = time.perf_counter()
    dataset.shuffle() # Shuffling

    # Train Step Loop
    for step_index, inputs in enumerate(dataset):

        with tf.GradientTape() as tape:
            model_output = model(inputs, is_train=True)

            trainable_variables = model.trainable_variables
            grads = tape.gradient(model_output['loss'], trainable_variables)
            optimizer.apply_gradients(zip(grads, trainable_variables))

            total_loss = model_output['loss'].numpy().mean()
            recon_loss = model_output['reconstr_loss'].numpy().mean()
            latent_loss = model_output['latent_loss'].numpy().mean()


            train_losses.append(total_loss)
            train_recon_errors.append(recon_loss)
            train_latent_losses.append(latent_loss)

            if step_index + 1 == n_batches:
                break

    loss.append(np.mean(train_losses, 0))
    reconstruct_loss.append(np.mean(train_recon_errors, 0))
    latent_loss.append(np.mean(train_latent_losses, 0))

    exec_time = time.perf_counter() - start_time
    print("Execution time: %0.3f \t Epoch %i: loss %0.4f | reconstr loss %0.4f | latent loss %0.4f"
          % (exec_time, epoch, loss[epoch], reconstruct_loss[epoch], latent_loss[epoch]))


    # Save progress every 5 epochs
    if (epoch + 1) % 5 == 0:
        checkpoint.save(save_prefix + "_" + str(epoch + 1))
        print("Model saved:", save_prefix)

# Save the final model
checkpoint.save(save_prefix)
print("Model saved:", save_prefix)